In [1]:
!pip install -U langchain_community --quiet
!pip install pypdf --quiet
!pip install instructor --quiet
!pip install faiss-cpu

In [ ]:
import os, json, glob
import torch
import requests
import numpy as np
from tqdm import tqdm
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from embeddings_wrapper import LangchainSentenceTransformer
from typing import Any
from pydantic import BaseModel, ValidationError, Field
from openai import AsyncAzureOpenAI


## Seleccionar parámetros en la siguiente celda

In [ ]:
from configs.maxi_config import API_KEY, ENDPOINT, MODEL, DEPLOYMENT # Cambiar 'maxi_config' por la que corresponda

JSONS_FOLDER = "datasets/fallos_json"
OUTPUT_FOLDER = "datasets/fallos_vectorstore"

EMBEDDINGS_MODEL = "mixedbread-ai/mxbai-embed-large-v1"
CHUNK_SIZE = 1024
CHUNK_OVERLAP = 128

In [ ]:
# Lee múltiples fallos en JSON
print("[INFO] Cargando y dividiendo los fallos JSON...")
splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

chunks = []

for path in glob.glob(os.path.join(JSONS_FOLDER, "*.json")):
    with open(path, encoding="utf-8") as f:
        fallos = json.load(f)          # cada archivo es un array con 1 fallo

    for fallo in fallos:
        contenido = fallo.get("CONTENIDO", {})
        for seccion, parrafos in contenido.items():
            texto = "\n".join(parrafos)            # solo esa sección
            for frag in splitter.split_text(texto):
                chunks.append(
                    Document(
                        page_content = frag,
                        metadata = {
                            "seccion": seccion,
                            "fuente" : os.path.basename(path)
                        }
                    )
                )

print(f"[INFO] Total de fragmentos generados: {len(chunks)}")

# --- Embeddings y FAISS ---
print("[INFO] Generando embeddings...")
embedding_model = LangchainSentenceTransformer()
texts, embeddings = embedding_model.embed_documents(chunks)


[INFO] Cargando y dividiendo los fallos JSON...
[INFO] Total de fragmentos generados: 253
[INFO] Generando embeddings...
[INFO] Filtering 253 chunks...
[INFO] Embedding 253 clean chunks...


Batches: 100%|██████████| 8/8 [01:34<00:00, 11.79s/it]


In [7]:
text_embedding_pairs = list(zip(texts, embeddings))
vectorstore = FAISS.from_embeddings(text_embedding_pairs, embedding_model)
vectorstore.save_local("datasets/fallos_vectorstore")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [8]:
def create_context(query):
    results = vectorstore.similarity_search(query, k=5)
    # for i, doc in enumerate(results, 1):
    #     print(f"{i}. {doc.page_content.strip()}\n")
    return results

azure_client = AsyncAzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint=ENDPOINT,
    api_key=API_KEY
)

async def get_response(query):
    res = await azure_client.chat.completions.create(
        model=DEPLOYMENT,
        messages=[
            {
                "role": "system",
                "content": "Sos un asistente judicial especializado en encontrar semejanzas entre fallos y responder preguntas sobre ellos. Tu tarea es analizar el contexto y responder de manera precisa y concisa.",
            },
            {
                "role": "system", 
                "content": f"El contexto es: {create_context(query)}"
            },
            {
                "role": "user",
                "content": query
            }
        ],
        
    )
    return res

In [13]:
response = await get_response("Explicame el fallo que dice: TERENZANO, ALCIDES VICENTE C/ LANDIVAR, MARTA OLGA Y OTRO S/ ORDINARIO COBRO DE PESOS -  Expte. Nº 8142")



In [14]:
respuesta = response.choices[0].message.content
respuesta


'El fallo mencionado se refiere a un caso judicial en el que Alcides Vicente Terenzano demanda a Marta Olga Landivar y a otro co-demandado en el marco de un "ordinario cobro de pesos". Este tipo de procedimiento generalmente se utiliza para reclamar el pago de deudas u obligaciones económicas.\n\nEn el contexto del expediente Nº 8142, se menciona específicamente una solicitud de regulación de honorarios por parte de los abogados involucrados en el caso, lo cual indica que ya hubo actuaciones profesionales previas relacionadas con el asunto. Se hace referencia a decisiones anteriores sobre recursos de inaplicabilidad de ley, lo que sugiere que ha habido un análisis sobre la aplicación de normas jurídicas en este caso, resueltas en fechas anteriores (25/6/2020 y 10/4/2023).\n\nAdemás, se hace alusión a la normativa arancelaria aplicable, destacando ciertos artículos de la ley 7046 que regulan cómo se deben establecer los honorarios de los abogados en estos casos. Esto sugiere una etapa e

In [15]:
from IPython.display import Markdown
display(Markdown(respuesta))


El fallo mencionado se refiere a un caso judicial en el que Alcides Vicente Terenzano demanda a Marta Olga Landivar y a otro co-demandado en el marco de un "ordinario cobro de pesos". Este tipo de procedimiento generalmente se utiliza para reclamar el pago de deudas u obligaciones económicas.

En el contexto del expediente Nº 8142, se menciona específicamente una solicitud de regulación de honorarios por parte de los abogados involucrados en el caso, lo cual indica que ya hubo actuaciones profesionales previas relacionadas con el asunto. Se hace referencia a decisiones anteriores sobre recursos de inaplicabilidad de ley, lo que sugiere que ha habido un análisis sobre la aplicación de normas jurídicas en este caso, resueltas en fechas anteriores (25/6/2020 y 10/4/2023).

Además, se hace alusión a la normativa arancelaria aplicable, destacando ciertos artículos de la ley 7046 que regulan cómo se deben establecer los honorarios de los abogados en estos casos. Esto sugiere una etapa en el proceso judicial relacionada con los costos y compensaciones por las actuaciones legales realizadas.

En resumen, el fallo en cuestión trata sobre un reclamo financiero, donde se regula la compensación por servicios legales en un contexto donde ya han sucedido actuaciones y resoluciones anteriores.